<a href="https://colab.research.google.com/github/kjinb1212/Falldown-detection-KD/blob/main/student_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.model_selection import train_test_split

from glob import glob
import os
import sys
from PIL import Image
import pandas as pd

In [ ]:
def train(model, train_loader, test_loader, criterion, optimizer, epochs, save_name):
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer, patience=4, verbose=True)
    best_loss = None
    best_acc = None
    patience = 0 
    
    history = {'loss': [], 'acc': []}
    
    for epoch in range(epochs):
        print("--------- epoch : {} ------------".format(epoch+1))
        model.train()
        train_losses = []
        for data, label in train_loader:
            data = data.to(device)
            label = label.to(device)
            
            optimizer.zero_grad()
            output = model(data)
                        
            loss = criterion(output, label)
            train_losses.append(loss.item())
            loss.backward()
            optimizer.step()
            torch.cuda.empty_cache()
        train_loss = np.average(train_losses)
        print("train loss: {}".format(train_loss))
        
        model.eval()
        test_losses = []
        correct = 0
        total = 0
        with torch.no_grad():
            for data, label in test_loader:
                data = data.to(device)
                label = label.to(device)
                
                output = model(data)
                loss = criterion(output, label)
                test_losses.append(loss.item())
                _, predict = torch.max(output.data, 1)
                correct += (predict == label).sum().item()
                total += label.size(0)
            test_loss = np.average(test_losses)
            test_acc = 100 * correct / total
            print("test loss: {}, \ttest acc: {}%".format(test_loss, test_acc))
            
            history['loss'].append(test_loss)
            history['acc'].append(test_acc)
            
            if (best_loss is None) or (best_loss > test_loss):
                best_loss = test_loss
                best_acc = test_acc
                torch.save(model.state_dict(), '3_model_weights/' + save_name + '.pth')
                print('Best loss: {}'.format(best_loss))
                patience = 0
            else:
                patience += 1

            if patience > 7:
                print("early stop at {} epoch".format(epoch + 1))
                break

            scheduler.step(metrics=test_loss)
    
    print("best loss: {},\t best acc: {}%".format(best_loss, best_acc))
    return best_loss, best_acc, history 

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, input_size, train = True, padding = True, normalize = False,
                 bright_ness = 0.2, hue = 0.15, contrast = 0.15, random_Hflip = 0.3, rotate_deg = 20):
        orig_normal_path = glob(os.path.join(root_dir, 'normal') + '/*.jpg')
        orig_fall_path = glob(os.path.join(root_dir, 'falldown') + '/*.jpg')
        orig_back_path = glob(os.path.join(root_dir, 'background') + '/*.jpg')
        
        normal_paths = []
        fall_paths = []
        back_paths = []
        
        for path in orig_normal_path:
            img = Image.open(path)
            if min(img.size[0], img.size[1]) < 32:
                pass
            else:
                normal_paths.append(path)
                
        for path in orig_fall_path:
            img = Image.open(path)
            if min(img.size[0], img.size[1]) < 32:
                pass
            else:
                fall_paths.append(path)
        
        for path in orig_back_path:
            img = Image.open(path)
            if min(img.size[0], img.size[1]) < 32:
                pass
            else:
                back_paths.append(path)
                        
        self.total_paths = normal_paths + fall_paths + back_paths
        self.labels = [0] * len(normal_paths) + [1] * len(fall_paths) + [2] * len(back_paths)
        
        transform = []
        if train:
            #transform.append(torchvision.transforms.ColorJitter(brightness=bright_ness, hue=hue, contrast=contrast))
            transform.append(torchvision.transforms.RandomHorizontalFlip(p=random_Hflip))
            #transform.append(torchvision.transforms.RandomCrop(224))
            transform.append(torchvision.transforms.RandomRotation(degrees=rotate_deg))
        transform.append(torchvision.transforms.ToTensor())
        if normalize:
            transform.append(torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]))
        if padding:
            transform.append(lambda x: torchvision.transforms.Pad(((128 - x.shape[2]) // 2, (128 - x.shape[1]) // 2), fill=0,
                                                     padding_mode="constant")(x))
        transform.append(torchvision.transforms.Resize((input_size, input_size)))
        self.transform = torchvision.transforms.Compose(transform)
        
        
    def __len__(self):
        return len(self.total_paths)

    def __getitem__(self, index):
        img = Image.open(self.total_paths[index])
        img = self.transform(img)
        return img, self.labels[index]

In [ ]:
class CNN_layers(nn.Module):
    def __init__(self):
        super(CNN_layers, self).__init__()      
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.conv4 = nn.Conv2d(64, 32, 3)
        #self.conv5 = nn.Conv2d(32, 16, 3)

        self.fc1 = nn.Linear(32 * 8 * 8, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 3)

        self.bn1 = nn.BatchNorm2d(16)
        self.bn2 = nn.BatchNorm2d(32)
        self.bn3 = nn.BatchNorm2d(64)
        self.bn4 = nn.BatchNorm2d(32)
        #self.bn5 = nn.BatchNorm2d(16)

        self.bn6 = nn.BatchNorm1d(16)
        self.bn7 = nn.BatchNorm1d(8)
        self.padding = nn.ZeroPad2d(1)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(self.padding(x))))) # 128 -> 64
        x = self.pool(F.relu(self.bn2(self.conv2(self.padding(x))))) # 64 -> 32
        x = self.pool(F.relu(self.bn3(self.conv3(self.padding(x))))) # 32 -> 16
        x = self.pool(F.relu(self.bn4(self.conv4(self.padding(x))))) # 16 -> 8
        #x = self.pool(F.relu(self.bn5(self.conv5(self.padding(x))))) # 8 -> 4

        x = x.view(-1, 32 * 8 * 8)
        x = F.relu(self.bn6(self.fc1(x)))
        x = F.relu(self.bn7(self.fc2(x)))
        x = self.fc3(x)
        return x

In [ ]:
INPUT_SIZE = 128
PADDING = False
NORMALIZE = False
BATCHSIZE = 128
NUMEPOCH = 100

train_data = CustomDataset(
    root_dir='train',
    input_size=INPUT_SIZE, train=True, padding=PADDING, normalize=NORMALIZE,
    bright_ness=0, hue=01.5, contrast=0.15, random_Hflip=0, rotate_deg=0)

test_data = CustomDataset(
    root_dir='validation',
    input_size=INPUT_SIZE, train=False, padding=PADDING, normalize=NORMALIZE,
    bright_ness=0, hue=01.5, contrast=0.15, random_Hflip=0, rotate_deg=0)

In [ ]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCHSIZE, shuffle=True, num_workers=32, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCHSIZE, shuffle=False, num_workers=32, drop_last=True)

In [ ]:
device = torch.device("cuda:3" if torch.cuda.is_available() else "cpu")
student_model = CNN_layers().to(device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(student_model.parameters(), weight_decay=1e-4, lr=0.001)
save_name = 'student_model'
loss, acc, history = train(model=student_model, train_loader=train_loader, test_loader=test_loader,
      criterion = criterion, optimizer=optimizer, epochs=NUMEPOCH, save_name=save_name) 

--------- epoch : 1 ------------
train loss: 0.38461847586451836
test loss: 0.25732003649075824, 	test acc: 89.0625%
Best loss: 0.25732003649075824
--------- epoch : 2 ------------
train loss: 0.21647600646851198
test loss: 0.2187693528831005, 	test acc: 92.70833333333333%
Best loss: 0.2187693528831005
--------- epoch : 3 ------------
train loss: 0.17334961725293466
test loss: 0.08508630096912384, 	test acc: 96.61458333333333%
Best loss: 0.08508630096912384
--------- epoch : 4 ------------
train loss: 0.14590217735407487
test loss: 0.06107185781002045, 	test acc: 97.65625%
Best loss: 0.06107185781002045
--------- epoch : 5 ------------
train loss: 0.1285243919997845
test loss: 0.14298561618973812, 	test acc: 95.57291666666667%
--------- epoch : 6 ------------
train loss: 0.11037975791490302
test loss: 0.16437064918378988, 	test acc: 94.79166666666667%
--------- epoch : 7 ------------
train loss: 0.09228940913980861
test loss: 0.21521655035515627, 	test acc: 93.48958333333333%
---------

In [ ]:
print("loss: {}\nacc: {}%".format(loss, acc))

loss: 0.06107185781002045
acc: 97.65625%


In [ ]:
df = pd.DataFrame(history)
df.to_csv("3_history/"+ save_name+ "_history.csv", mode='w')